In [71]:
#Import Librairies
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe" # to be replaced by "iframe" if working on JULIE

In [72]:
#Import dataset
dataset = pd.read_csv('cancer_reg.csv')

In [73]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

dataset.info()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 3047

Display of dataset: 


,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657



Basics statistics: 


,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
count,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3.047000e+03,3047.000000,3047.000000,3047,3047.000000,...,2438.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(54545.6, 61494.5]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,606.338544,185.965868,178.664063,448.268586,47063.281917,1.026374e+05,16.878175,155.399415,NaN,45.272333,...,48.453774,41.196324,36.252642,19.240072,83.645286,9.107978,1.253965,1.983523,51.243872,5.640306
std,1416.356223,504.134286,27.751511,54.560733,12040.090836,3.290592e+05,6.409087,529.628366,NaN,45.304480,...,10.083006,9.447687,7.841741,6.113041,16.380025,14.534538,2.610276,3.517710,6.572814,1.985816
min,6.000000,3.000000,59.700000,201.300000,22640.000000,8.270000e+02,3.200000,0.000000,NaN,22.300000,...,15.700000,13.500000,11.200000,2.600000,10.199155,0.000000,0.000000,0.000000,22.992490,0.000000
25%,76.000000,28.000000,161.200000,420.300000,38882.500000,1.168400e+04,12.150000,0.000000,NaN,37.700000,...,41.000000,34.500000,30.900000,14.850000,77.296180,0.620675,0.254199,0.295172,47.763063,4.521419
50%,171.000000,61.000000,178.100000,453.549422,45207.000000,2.664300e+04,15.900000,0.000000,NaN,41.000000,...,48.700000,41.100000,36.300000,18.800000,90.059774,2.247576,0.549812,0.826185,51.669941,5.381478
75%,518.000000,149.000000,195.200000,480.850000,52492.000000,6.867100e+04,20.400000,83.650776,NaN,44.000000,...,55.600000,47.700000,41.550000,23.100000,95.451693,10.509732,1.221037,2.177960,55.395132,6.493677



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avganncount              3047 non-null   float64
 1   avgdeathsperyear         3047 non-null   int64  
 2   target_deathrate         3047 non-null   float64
 3   incidencerate            3047 non-null   float64
 4   medincome                3047 non-null   int64  
 5   popest2015               3047 non-null   int64  
 6   povertypercent           3047 non-null   float64
 7   studypercap              3047 non-null   float64
 8   binnedinc                3047 non-null   object 
 9   medianage                3047 non-null   float64
 10  medianagemale            3047 non-null   float64
 11  medianagefemale          3047 non-null   float64
 12  geography                3047 non-null   object 
 13  percentmarried           3047 non-null   float64
 14  pctnohs18_24           

avganncount                 0.000000
avgdeathsperyear            0.000000
target_deathrate            0.000000
incidencerate               0.000000
medincome                   0.000000
popest2015                  0.000000
povertypercent              0.000000
studypercap                 0.000000
binnedinc                   0.000000
medianage                   0.000000
medianagemale               0.000000
medianagefemale             0.000000
geography                   0.000000
percentmarried              0.000000
pctnohs18_24                0.000000
pcths18_24                  0.000000
pctsomecol18_24            74.991795
pctbachdeg18_24             0.000000
pcths25_over                0.000000
pctbachdeg25_over           0.000000
pctemployed16_over          4.988513
pctunemployed16_over        0.000000
pctprivatecoverage          0.000000
pctprivatecoveragealone    19.986872
pctempprivcoverage          0.000000
pctpubliccoverage           0.000000
pctpubliccoveragealone      0.000000
p

In [74]:
#Delete the na values
dataset.dropna()

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
1,173.000000,70,161.3,411.600000,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
3,427.000000,202,194.8,430.400000,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.000000,26,144.4,350.100000,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657
7,146.000000,71,183.6,404.000000,40189,20848,17.8,0.000000,"(37413.8, 40362.7]",51.7,...,33.1,25.9,50.9,24.1,89.406636,0.305159,1.889077,2.286268,48.967033,5.889179
14,2265.000000,901,171.0,440.700000,50083,490945,16.3,462.373586,"(48021.6, 51046.4]",37.2,...,50.6,42.5,36.5,21.4,89.038167,1.827041,2.315986,1.033625,48.188377,5.355836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,1962.667684,7,209.6,453.549422,45353,1843,11.4,0.000000,"(45201, 48021.6]",45.7,...,52.7,43.9,32.2,9.2,97.960199,0.000000,0.547264,0.995025,57.780612,4.664723
3034,1962.667684,85,184.4,453.549422,45180,35788,15.0,1900.078239,"(42724.4, 45201]",38.2,...,52.3,46.2,34.4,17.4,87.718617,3.168048,1.610355,1.893068,50.730567,7.404551
3043,1962.667684,43,150.1,453.549422,48609,37118,18.8,377.175494,"(48021.6, 51046.4]",30.4,...,53.3,48.6,28.8,17.7,75.706245,2.326771,4.044920,14.130288,52.007937,8.186470
3044,1962.667684,46,153.9,453.549422,51144,34536,15.0,1968.959926,"(51046.4, 54545.6]",30.9,...,52.6,47.8,26.6,16.8,87.961629,2.313188,1.316472,5.680705,55.153949,7.809192


In [75]:
#Check  na deletion
dataset.isna().sum()

avganncount                   0
avgdeathsperyear              0
target_deathrate              0
incidencerate                 0
medincome                     0
popest2015                    0
povertypercent                0
studypercap                   0
binnedinc                     0
medianage                     0
medianagemale                 0
medianagefemale               0
geography                     0
percentmarried                0
pctnohs18_24                  0
pcths18_24                    0
pctsomecol18_24            2285
pctbachdeg18_24               0
pcths25_over                  0
pctbachdeg25_over             0
pctemployed16_over          152
pctunemployed16_over          0
pctprivatecoverage            0
pctprivatecoveragealone     609
pctempprivcoverage            0
pctpubliccoverage             0
pctpubliccoveragealone        0
pctwhite                      0
pctblack                      0
pctasian                      0
pctotherrace                  0
pctmarri

In [76]:
# Drop useless columns / columns with too many missing values
useless_cols = ['pctemployed16_over','pctprivatecoveragealone','pctsomecol18_24','avganncount','avgdeathsperyear','popest2015','medianagemale','medianagefemale','geography','pctwhite','pctblack','pctasian','pctotherrace','pctmarriedhouseholds','binnedinc']

print("Dropping useless columns...")
dataset = dataset.drop(useless_cols, axis=1)
print("...Done.")
print(dataset.head())

Dropping useless columns...
...Done.
   target_deathrate  incidencerate  medincome  povertypercent  studypercap  \
0             164.9          489.8      61898            11.2   499.748204   
1             161.3          411.6      48127            18.6    23.111234   
2             174.7          349.7      49348            14.6    47.560164   
3             194.8          430.4      44243            17.1   342.637253   
4             144.4          350.1      49955            12.5     0.000000   

   medianage  percentmarried  pctnohs18_24  pcths18_24  pctbachdeg18_24  \
0       39.3            52.5          11.5        39.5              6.9   
1       33.0            44.5           6.1        22.4              7.5   
2       45.0            54.2          24.0        36.6              9.5   
3       42.8            52.7          20.2        41.2              2.5   
4       48.3            57.8          14.9        43.0              2.0   

   pcths25_over  pctbachdeg25_over  pctunem

In [77]:
dataset.isna().sum()

target_deathrate          0
incidencerate             0
medincome                 0
povertypercent            0
studypercap               0
medianage                 0
percentmarried            0
pctnohs18_24              0
pcths18_24                0
pctbachdeg18_24           0
pcths25_over              0
pctbachdeg25_over         0
pctunemployed16_over      0
pctprivatecoverage        0
pctempprivcoverage        0
pctpubliccoverage         0
pctpubliccoveragealone    0
birthrate                 0
dtype: int64

In [78]:
# Separate target variable Y from features X
target_name = 'target_deathrate'

print("Separating labels from features...")
Y = dataset.loc[:,target_name]
X = dataset.loc[:,[c for c in dataset.columns if c!=target_name]] # Keeping all columns
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    164.9
1    161.3
2    174.7
3    194.8
4    144.4
Name: target_deathrate, dtype: float64

   incidencerate  medincome  povertypercent  studypercap  medianage  \
0          489.8      61898            11.2   499.748204       39.3   
1          411.6      48127            18.6    23.111234       33.0   
2          349.7      49348            14.6    47.560164       45.0   
3          430.4      44243            17.1   342.637253       42.8   
4          350.1      49955            12.5     0.000000       48.3   

   percentmarried  pctnohs18_24  pcths18_24  pctbachdeg18_24  pcths25_over  \
0            52.5          11.5        39.5              6.9          23.2   
1            44.5           6.1        22.4              7.5          26.0   
2            54.2          24.0        36.6              9.5          29.0   
3            52.7          20.2        41.2              2.5          31.6   
4            57.8          14.9        43.0 

In [79]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X = X.values
Y = Y.tolist()
print("...Done")
print(X[0:5,:])
print()
print(Y[0:5])

Convert pandas DataFrames to numpy arrays...
...Done
[[4.89800000e+02 6.18980000e+04 1.12000000e+01 4.99748204e+02
  3.93000000e+01 5.25000000e+01 1.15000000e+01 3.95000000e+01
  6.90000000e+00 2.32000000e+01 1.96000000e+01 8.00000000e+00
  7.51000000e+01 4.16000000e+01 3.29000000e+01 1.40000000e+01
  6.11883103e+00]
 [4.11600000e+02 4.81270000e+04 1.86000000e+01 2.31112344e+01
  3.30000000e+01 4.45000000e+01 6.10000000e+00 2.24000000e+01
  7.50000000e+00 2.60000000e+01 2.27000000e+01 7.80000000e+00
  7.02000000e+01 4.36000000e+01 3.11000000e+01 1.53000000e+01
  4.33309558e+00]
 [3.49700000e+02 4.93480000e+04 1.46000000e+01 4.75601636e+01
  4.50000000e+01 5.42000000e+01 2.40000000e+01 3.66000000e+01
  9.50000000e+00 2.90000000e+01 1.60000000e+01 7.00000000e+00
  6.37000000e+01 3.49000000e+01 4.21000000e+01 2.11000000e+01
  3.72948782e+00]
 [4.30400000e+02 4.42430000e+04 1.71000000e+01 3.42637253e+02
  4.28000000e+01 5.27000000e+01 2.02000000e+01 4.12000000e+01
  2.50000000e+00 3.160000

In [80]:
#Splitting dataset into train set and test set
from sklearn.model_selection import train_test_split
print("Dividing into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [81]:
#import stardardisation librairy
from sklearn.preprocessing import StandardScaler

In [82]:
#Preprocessing
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)

X_test = sc_x.transform(X_test)

In [83]:
#Creating instances
lin = LinearRegression()
lasso1 = Lasso(alpha = 1)
lasso001 = Lasso(alpha = 0.01)
lasso00001 = Lasso(alpha = 0.0001)

In [84]:
#Fitting
lin.fit(X_train, y_train)
lasso1.fit(X_train, y_train)
lasso001.fit(X_train, y_train)
lasso00001.fit(X_train, y_train)

Lasso(alpha=0.0001)

In [85]:
#Trains scores
print("Train scores \n")
print("Score Lin : {} \nScore Lasso1 : {} \nScore Lasso001 : {} \nScore Lasso00001 : {}".format(lin.score(X_train, y_train),lasso1.score(X_train, y_train),
                                                                                                       lasso001.score(X_train, y_train),lasso00001.score(X_train,y_train)))

Train scores 

Score Lin : 0.4808325694370772 
Score Lasso1 : 0.47061286623844034 
Score Lasso001 : 0.4808285420906634 
Score Lasso00001 : 0.48083256903296123


In [86]:
#Tests scores
print("Test scores \n")
print("Score Lin : {} \nScore Lasso1 : {} \nScore Lasso001 : {} \nScore Lasso00001 : {}".format(lin.score(X_test, y_test),
lasso1.score(X_test, y_test),lasso001.score(X_test, y_test),lasso00001.score(X_test,y_test)))

Test scores 

Score Lin : 0.546720076839537 
Score Lasso1 : 0.5308885770888854 
Score Lasso001 : 0.5466798701213592 
Score Lasso00001 : 0.5467197164425455


In [87]:
#X_Train Predictions
y_train_pred_lin = lin.predict(X_train)
y_train_pred_l1 = lasso1.predict(X_train)
y_train_pred_l001 = lasso001.predict(X_train)
y_train_pred_l00001 = lasso00001.predict(X_train) 

In [88]:
#Train mean squared errors
print("Train MSE \n")
print("MSE Lin : {} \nMSE Lasso1 : {} \nMSE Lasso001 : {} \nMSE Lasso00001 : {}".format(mean_squared_error(y_train, y_train_pred_lin),mean_squared_error(y_train, y_train_pred_l1),
mean_squared_error(y_train, y_train_pred_l001),
mean_squared_error(y_train, y_train_pred_l00001)))

Train MSE 

MSE Lin : 397.0177041619777 
MSE Lasso1 : 404.83291532948147 
MSE Lasso001 : 397.02078395422376 
MSE Lasso00001 : 397.0177044710132


In [89]:
#X_test predictions
y_test_pred_lin = lin.predict(X_test)
y_test_pred_l1 = lasso1.predict(X_test)
y_test_pred_l001 = lasso001.predict(X_test)
y_test_pred_l00001 = lasso00001.predict(X_test)

In [90]:
#Test mean squared errors
print("Test MSE \n")
print("MSE Lin : {} \nMSE Lasso1 : {} \nMSE Lasso001 : {} \nMSE Lasso00001 : {}".format(mean_squared_error(y_test, y_test_pred_lin),
mean_squared_error(y_test,lin.predict(X_test)),
mean_squared_error(y_test,lasso1.predict(X_test)),
mean_squared_error(y_test,lasso001.predict(X_test)),
mean_squared_error(y_test,lasso00001.predict(X_test))))

Test MSE 

MSE Lin : 357.6261781879693 
MSE Lasso1 : 357.6261781879693 
MSE Lasso001 : 370.1168235078262 
MSE Lasso00001 : 357.65790025247003


In [91]:
print("Linear Regression \n Proportion of coefficients equal to zero")
print(np.sum(lin.coef_==0)/len(lin.coef_))
print("\n Distribution of coefficients \n")
px.histogram(lin.coef_)

Linear Regression 
 Proportion of coefficients equal to zero
0.0

 Distribution of coefficients 



In [92]:
print("Lasso 1 \n Proportion of coefficients equal to zero")
print(np.sum(lasso1.coef_==0)/len(lasso1.coef_))
print("\n Distribution of coefficients \n")
px.histogram(lasso1.coef_)

Lasso 1 
 Proportion of coefficients equal to zero
0.4117647058823529

 Distribution of coefficients 



In [93]:
print("Lasso 0.01 \n Proportion of coefficients equal to zero")
print(np.sum(lasso001.coef_==0)/len(lasso001.coef_))
print("\n Distribution of coefficients \n")
px.histogram(lasso001.coef_)

Lasso 0.01 
 Proportion of coefficients equal to zero
0.0

 Distribution of coefficients 



In [94]:
print("Lasso 0.0001 \n Proportion of coefficients equal to zero")
print(np.sum(lasso00001.coef_==0)/len(lasso00001.coef_))
print("\n Distribution of coefficients \n")
px.histogram(lasso00001.coef_)

Lasso 0.0001 
 Proportion of coefficients equal to zero
0.0

 Distribution of coefficients 



The Lasso model with alpha = 1 is the one with the more Proportion of coefficients equal to zero compared to those ones with an alpha = 0.01 and alpha = 0.0001. There must be a value of alpha that can be suitable for the Lasso model that can allow us to obtain the most Proportion of coefficients equal to zero. We are going to perform a gridSearch to find this value.

In [25]:
params = {'alpha' : [10**(-a) for a in range(10)]}
lasso = Lasso()
grid = GridSearchCV(lasso,param_grid=params, cv = 10, verbose=1)

grid.fit(X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06,
                                   1e-07, 1e-08, 1e-09]},
             verbose=1)

In [26]:
grid.best_params_

{'alpha': 0.1}

In [30]:
#Summary
print("BEST ESTIMATOR \n")
print("train R^2 score \n")
best_model = grid.best_estimator_
print(best_model.score(X_train, y_train))
print("\n")
print("test R^2 score \n")
print(best_model.score(X_test, y_test))
print("\n \n")
print("train MSE \n")
print(mean_squared_error(y_train,best_model.predict(X_train)))
print("\n")
print("test MSE \n")
print(mean_squared_error(y_test,best_model.predict(X_test)))

BEST ESTIMATOR 

train R^2 score 

0.4804350080679466


test R^2 score 

0.5460008945414773

 

train MSE 

397.3217272819653


test MSE 

358.1935944875535


We have obtain the same results but the model with Lasso depends on less explicative features